<i><b>Public AI</b></i>
<br>

# 심층 신경망 분류기 만들기


### _Objective_
1. 텐서플로우를 활용해 심층 신경망 분류기를 만들어봅니다. 
2. 하이퍼파라미터를 결정하여 모델을 학습시켜봅니다. 

In [ ]:
%matplotlib inline

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tensorflow.keras.utils import get_file

### 예제 데이터 ) 암 환자 데이터 셋

나이와 종양크기를 바탕으로 암 환자의 종양이 양성인지 음성인지를 분류하는 데이터셋입니다.

In [ ]:
fpath = get_file("cancer_dataset.csv",
                 "https://s3.ap-northeast-2.amazonaws.com/pai-datasets/alai-deeplearning/cancer_dataset.csv")
cancer_df = pd.read_csv(fpath)
X = cancer_df[['age','tumor_size']]
y = cancer_df["label"]

X = (X - X.min()) / (X.max() - X.min())

### 데이터 시각화
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(1,1,1)

# 0: 정상
X[y==0].plot('age','tumor_size',ax=ax,legend=True,
                           kind='scatter',color='red')
# 1: 암환자
X[y==1].plot('age','tumor_size',ax=ax,legend=True,
                           kind='scatter',color='green')
plt.legend(['Normal','Cancer'])
plt.title("Cancer Or Not")
plt.show()

## 1. Network Model 생성하기



앞서 심층 신경망 모델의 개념을 배우며 텐서플로우를 이용해 간략하게 1번째 층의 유닛이 10개인 신경망 모델을 만들어봤습니다. 이번에는 1번째 층의 유닛이 200개인 신경망을 만들어봅시다. 

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

In [ ]:
# 입력층 구현
inputs = #

# 은닉층 구현
hidden = #

# 출력층 구현
output = #

# 모델 구현
model = #

## 2. 학습을 위해 Model 학습과정 설정하기

keras에서는 `compile()`함수를 사용해 학습하기 전에 학습에 대한 설정을 수행합니다. (1) 최적화 함수, (2) 손실 함수, (3) 평가 지표 등을 결정하여 모델을 준비하는 방법을 알아보고, (4) 준비된 최적화 함수, 손실 함수, 평가지표 등으로 구성된 모델을 컴파일하는 방법을 알아봅시다. 

### (1) 최적화 함수 결정하기

신경망 학습의 목적은 Loss Function의 결과값을 최소화하는 매개변수를 찾는 것입니다. 즉, 매개변수의 최적값을 찾는 것이 신경망 학습의 주된 문제이며 이러한 문제를 푸는 것을 **최적화**라고 합니다.최적화를 도와주는 함수로는 `SGD`, `momentum`, `NAG`, `Adagrad`, `RMSprop`, `Adam`등이 있습니다. 

keras에서는 아래와 같이 최적화 함수를 적용할 수 있습니다.

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop, Adam

In [ ]:
optim = SGD(learning_rate=1e-2)
optim = RMSprop(learning_rate=1e-2, rho=0.9)
optim = Adam(learning_rate=1e-2, beta_1=0.9, beta_2=0.999)

최적화 함수와 같이 직접 설정해주어야 하는 매개변수를 **하이퍼파라미터**라고 합니다. 하이퍼파라미터는 편향(bias)나 가중치(weight) 같은 신경망의 매개변수와는 성질이 다른 매개변수입니다. 신경망의 가중치는 훈련데이터와 학습 알고리즘에 의해서 자동으로 획득되는 매개변수인 반면, 최적화 함수와 같은 하이퍼파라미터는 사람이 직접 설정해야 하는 매개변수입니다. 일반적으로 이 하이퍼파라미터들은 여러 후보 값 중에서 시험을 통해 가장 잘 학습하는 값을 찾는 과정을 거쳐야 하는데, 이 과정을 하이퍼파라미터 최적화라고 합니다.

### (2) 손실함수 결정하기

손실함수에는 MSE외에 mean_absolute_error, categorical_crossentropy, binary_crossentropy등의 종류가 있습니다. MAE(mean_absolute_errer)는 MSE에서 제곱 대신 절댓값을 취한 함수로 특이점에 크게 영향을 받지 않는다는 장점이 있습니다. 보통 classification문제를 해결할 때 crossentropy 함수들을 사용합니다. <br>

아래의 표는 모델에 맞는 마지막 층의 활성화 함수와 손실 함수 선택을 정리한 표입니다.

| 문제 유형 | 마지막 층의 활성화 함수 | 손실 함수 |
| --- | --- | --- |
| 이진 분류 | 시그모이드 | binary_crossentropy |
| 단일 label 다중 분류 | 소프트맥스 | categorical_crossentropy |
| 다중 label 다중 분류 | 시그모이드 | binary_crossentropy |
| 임의 값에 대한 회귀 | 없음 | mse |
| 0과 1 사이 값에 대한 회귀 | 시그모이드 | mse 또는 binary_crossentropy|

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import MeanSquaredError

loss_func = #

### (3)  평가 지표 결정하기

keras에서는 `tensorflow.keras.metrics`를 통해 평가 지표를 설정할 수 있습니다. 주요 평가지표로는 정확도를 나타내는 accuracy, binary_accuracy, categorical_accuracy, 상위 k개 예측값의 정확도를 나타내는 top_k_categorical_accuracy등이 있습니다.

In [ ]:
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import TopKCategoricalAccuracy

metric_func = BinaryAccuracy()

### (4) 모델 컴파일하기

keras의 `compile()` 과정에서 우리는 하이퍼파라미터인 최적화 함수, 손실함수, 평가 지표를 설정합니다. 이 과정에서 케라스의 기본 백엔드인 tensorflow에서는 최적화 함수, 손실함수, 평가 지표를 연결하여 계산 그래프 구성을 마무리하는 작업이 수행됩니다.

In [ ]:
model.compile(#
)

## 3. Model 학습시키기

keras에서는 `fit()`함수를 사용해 훈련셋을 이용하여 구성한 모델로 학습시킵니다.

`fit()` 함수의 주요 인자로는 epoch와 batch_size가 있습니다. 전체 훈련 데이터에 수행되는 각 반복을 **에포크(Epoch)**라고 합니다. 각 epoch마다 네트워크가 배치에서 손실에 대한 가중치의 그래디언트를 계산하고 그에 맞추어 가중치를 업데이트 합니다. batch_size는 기본 32로 설정되어 있습니다. 한 번에 32개의 데이터씩 처리한다는 의미입니다. 

In [ ]:
history = model.fit(#
)

한편, keras에서는 `history`함수를 사용해 모델 학습 시 훈련셋, 검증셋의 손실 및 정확도를 측정합니다. 반복횟수에 따른 손실 및 정확도 추이를 보면서 학습 상황을 판단합니다.

아래 그래프를 보면, `epoch = 1000`으로 설정함에 따라, 1000번 반복하여 모델을 훈련하는 과정에서 손실이 줄어드는 것을 확인할 수 있습니다. 

In [ ]:
plt.plot(history.history['loss'])
plt.show()

아래 그래프를 보면 1000번을 반복하여 모델을 훈련하는 과정에서 정확도가 올라가는 것을 확인할 수 있습니다.  

In [ ]:
plt.plot(history.history['binary_accuracy'])
plt.show()

## 4. 결과 확인하기

자, 이제 모델을 만들고 학습시켜 보았습니다. `results = model.predict(xs)`를 이용해 만들어진 모델에 데이터를 넣고 결과값을 예측해봅시다. 그래프를 보면, 단순히 직선으로 암 환자와 정상을 구분하지 않고, 다수의 곡선으로 경계가 나뉘어진 것을 확인할 수 있습니다. 

In [ ]:
x1s,x2s = np.mgrid[0:1:0.01,0:1:0.01]
xs = np.stack([x1s.ravel(),x2s.ravel()],axis=1)

results = model.predict(xs) # 결과 예측

colors = []
for result in results.ravel():
    if result < 0.5:
        colors.append('g')
    else:
        colors.append('r')
        
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(1,1,1)
ax.scatter(xs[:,0],xs[:,1],c=colors,alpha=0.1)

# 0: 정상
X[y==1].plot('age','tumor_size',ax=ax,legend=True,
                                   kind='scatter',color='red')
# 1: 암환자
X[y==0].plot('age','tumor_size',ax=ax,legend=True,
                                   kind='scatter',color='green')
plt.legend(["decision",'Normal','Cancer'])
plt.title("Cancer Or Not")
plt.show()

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/09/16


---